In [57]:
import jams
import numpy as np
import os
import pretty_midi
import random
np.int = int # deprecated np.int

In [44]:
string_names = ['e', 'B', 'G', 'D', 'A', 'E']

technique_symbols = {
    'bend': 'b',
    'slide': 's',
    'hammer on': 'h',
    'pull off': 'p',
    'vibrato': '~',
    'tapping': 'T',
    'sweep picking': 'S',
    'alternate picking': 'a',
    'legato': 'l'
}

In [58]:
def jam_to_tab(jam, notes_per_row=50):
    '''convert jams object into ascii tabs'''
    notes = []
    for ann in jam.annotations:
        if ann.namespace == "tab_note":  # custom namespace
            for obs in ann.data:
                val = obs.value  # expected: dict with 'string', 'fret', 'techniques'
                notes.append({
                    'time': obs.time,
                    'string': val['string'],  # 1-6 (1 = high e, 6 = low E)
                    'fret': val['fret'],
                    'techniques': val.get('techniques', [])
                })

    # sort notes by time
    notes.sort(key=lambda x: x['time'])

    # initialize output 
    tab_lines = [''] * 6
    rows = []
    counter = 0

    for note in notes:
        string_idx = note['string'] - 1  # 0 = high e
        fret_str = str(note['fret'])

        # add techniques as symbols
        for tech in note['techniques']:
            symbol = technique_symbols.get(tech, '')
            if symbol:
                fret_str += symbol

        # pad all strings equally to account for multiple techniques
        for i in range(6):
            if i == string_idx:
                tab_lines[5 - i] += fret_str.ljust(3, '-')  # pad for spacing
            else:
                tab_lines[5 - i] += '---'

        counter += 1
        if counter >= notes_per_row:
            # save row and reset
            row_block = "\n".join(f"{name}|{line}" for name, line in zip(string_names, tab_lines))
            rows.append(row_block)
            rows.append("")  
            tab_lines = [''] * 6
            counter = 0

    if any(tab_lines):
        row_block = "\n".join(f"{name}|{line}" for name, line in zip(string_names, tab_lines))
        rows.append(row_block)

    return "\n".join(rows)

In [59]:
# test with a sample jams
jam = jams.JAMS()
ann = jams.Annotation(namespace="tab_note")

ann.append(time=0.0, duration=0.5, value={'string': 1, 'fret': 3, 'techniques': ['bend']})
ann.append(time=0.5, duration=0.5, value={'string': 2, 'fret': 5, 'techniques': ['hammer on', 'vibrato']})
ann.append(time=1.0, duration=0.5, value={'string': 6, 'fret': 0, 'techniques': []})

jam.annotations.append(ann)

In [60]:
ascii_tab = jam_to_tab(jam, notes_per_row=10)
print(ascii_tab)

e|------0--
B|---------
G|---------
D|---------
A|---5h~---
E|3b-------


In [61]:
midi_dir= '/data/akshaj/MusicAI/GuitarSet/MIDIAnnotations/'

In [62]:
def midi_to_jams(midi_path):
    pm = pretty_midi.PrettyMIDI(midi_path)
    guitar_notes = pm.instruments[0].notes  # first instrument
    jam = jams.JAMS()

    note_ann = jams.Annotation(namespace='note')  # create a note annotation
    for note in guitar_notes:
        note_ann.append(
            time=note.start,
            duration=note.end - note.start,
            value=note.pitch,
            confidence=note.velocity / 127  # normalize velocity
        )

    jam.annotations.append(note_ann)
    return jam

In [63]:
def encode_notes_for_test(jam):
    '''adds sample random expressive techniques, random strings, and frets for testing/tab output purposes'''
    new_ann = jams.Annotation(namespace='tab_note')
    tech_options = ["slide", "vibrato", "hammer-on", "pull-off", "bend", None]

    # iterate over existing note 
    note_ann = jam.annotations[0]  
    for obs in note_ann.data:
        pitch = obs.value  # the original pitch

        value = {
            "pitch": pitch,
            "string": random.randint(1, 6),       # random string 1-6
            "fret": random.randint(0, 12),        # random fret 0-12
            "techniques": [random.choice(tech_options)] if random.random() < 0.5 else []
        }

        new_ann.append(time=obs.time, duration=obs.duration, value=value, confidence=obs.confidence)

    # add new tab_note annotation
    jam.annotations.append(new_ann)
    return jam

In [64]:
example_midi = os.path.join(midi_dir, '05_Funk2-108-Eb_solo.mid')

jam = midi_to_jams(example_midi)
jam = encode_notes_for_test(jam)
tab_output = jam_to_tab(jam, notes_per_row=20)
print(tab_output)

e|---------------------11----------------8--------------0-----
B|------7b-12s------------------4-----------10-------------3--
G|------------5-----------12-------------------3s-7~----------
D|12-11~---------------------1--------------------------------
A|---------------8--8--------------9~-5--------------7--------
E|------------------------------------------------------------

e|------------------------------------------8--9--------------
B|8~----------1b----0-----------------10-------------------12-
G|------6b-------12----11-------10-------4--------5b----------
D|---11----0--------------8-----------------------------------
A|------------------------------------------------------------
E|---------------------------6s----3-----------------2--6-----

e|------------------------------------------12-------0-----12-
B|---------------------------------5--------------------1b----
G|---------------12-------0-----------------------12s---------
D|------2--0--9-----1-----------------9--------3-----